In [1]:
import pandas as pd
pd.options.display.float_format = '{:.3f}'.format
pd.set_option("display.max_rows", None)  # 모든 행 출력
pd.set_option("display.max_columns", None)  # 모든 열 출력
import numpy as np
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB
from itertools import product
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from tqdm import tqdm
from functions import (load_parameters, load_generation_data, load_price_data, generate_randomized_generation,
generate_rt_scenarios, plot_generation_data, plot_randomized_generation, plot_scenarios_for_generator, plot_rt_scenarios, plot_summary)
from utils import optimize_hol, optimize_without, optimize_without_loop, compute_price_functions, transform_step

generation_data, I, T = load_generation_data(date_filter="2022-07-18")
S, R, P_RT, K, K0, M1, M2 = load_parameters(I, T, generation_data)
P_DA, P_PN = load_price_data()

✅ 총 5개 파일을 불러왔습니다: 1201.csv, 137.csv, 401.csv, 524.csv, 89.csv
📊 데이터 Shape: I=5, T=24, S=20
✅ 시뮬레이션 초기화 완료: S=20, Randomness='high', M1=757.00, M2=2111.00


#### Holistic Optimization

In [2]:
x_hol, a_hol, yp_hol, ym_hol, z_hol, zc_hol, zd_hol, ep_hol, bp_hol, em_hol, bm_hol, d_hol, dp_hol, dm_hol, obj_hol = optimize_hol(R, K, K0, P_DA, P_RT, P_PN, I, T, S, M1, M2)

Set parameter Username
Set parameter LicenseID to value 2611964
Academic license - for non-commercial use only - expires 2026-01-20
Set parameter MIPGap to value 1e-07
Optimal solution found! Objective value: 1737445.6648542627


#### Holistic Optimization without target_i

In [3]:
x_without, ep_without, em_without, yp_without, ym_without, d_without, dp_without, dm_without, i_map_without = optimize_without_loop(R, K, K0, P_DA, P_RT, P_PN, I, T, S)

Solving settlement model for each target DER:   0%|          | 0/5 [00:00<?, ?it/s]

Set parameter MIPGap to value 1e-07


Solving settlement model for each target DER:  20%|██        | 1/5 [00:07<00:31,  7.84s/it]

Set parameter MIPGap to value 1e-07


Solving settlement model for each target DER:  40%|████      | 2/5 [00:14<00:21,  7.22s/it]

...

Solving settlement model for each target DER:  60%|██████    | 3/5 [00:18<00:11,  5.66s/it]

Set parameter MIPGap to value 1e-07


Solving settlement model for each target DER:  80%|████████  | 4/5 [00:25<00:06,  6.05s/it]

Set parameter MIPGap to value 1e-07


Solving settlement model for each target DER: 100%|██████████| 5/5 [00:38<00:00,  7.71s/it]


#### Price Functions

In [86]:
rdc, rsc, rho_plus, rho_minus, total_demand_without, total_supply_without = compute_price_functions(
    yp_without, ym_without, dp_without, dm_without, i_map_without, P_RT, P_PN, T, S, I
)

In [90]:
target_i, t, s = 1, 16, 3

ap, bp = rho_plus[target_i, t, s, :]
am, bm = rho_minus[target_i, t, s, :]

print(f"ρ⁺(d) = {ap:.2f} + {bp:.2f}·d")
print(f"ρ⁻(d) = {am:.2f} + {bm:.2f}·d")

ρ⁺(d) = 225.16 + -0.44·d
ρ⁻(d) = 225.16 + 0.44·d


#### Transform Stepwise

In [91]:
rho_plus_step, rho_minus_step, B_map_plus, B_map_minus = transform_step(rho_plus, rho_minus, total_demand_without, total_supply_without)

In [96]:
print(yp_without[target_i][:, t, s])
print(rho_plus_step[target_i, t, s]\n)

print(ym_without[target_i][:, t, s])
print(rho_minus_step[target_i, t, s])

[  0.   0. 192.   0.]
[[  0.         225.15645144]
 [  0.50520833 224.93617509]
 [  1.51041667 224.49789328]
 [  2.515625   224.05961147]
 [  3.52083333 223.62132965]
 [  4.52604167 223.18304784]
 [  5.53125    222.74476603]
 [  6.53645833 222.30648422]
 [  7.54166667 221.8682024 ]
 [  8.546875   221.42992059]
 [  9.55208333 220.99163878]
 [ 10.55729167 220.55335697]
 [ 11.5625     220.11507515]
 [ 12.56770833 219.67679334]
 [ 13.57291667 219.23851153]
 [ 14.578125   218.80022971]
 [ 15.58333333 218.3619479 ]
 [ 16.58854167 217.92366609]
 [ 17.59375    217.48538428]
 [ 18.59895833 217.04710246]
 [ 19.60416667 216.60882065]
 [ 20.609375   216.17053884]
 [ 21.61458333 215.73225703]
 [ 22.61979167 215.29397521]
 [ 23.625      214.8556934 ]
 [ 24.63020833 214.41741159]
 [ 25.63541667 213.97912978]
 [ 26.640625   213.54084796]
 [ 27.64583333 213.10256615]
 [ 28.65104167 212.66428434]
 [ 29.65625    212.22600252]
 [ 30.66145833 211.78772071]
 [ 31.66666667 211.3494389 ]
 [ 32.671875   210.91

In [93]:
print(ym_without[1][:, 16, 3])

[192.   0.   0.   0.]
